---

**Config files**:
- Usually written in .yaml(yet another markup language) or .json(JavaScript Object Notation) or .xml(Extensible Markup Language) formats, config files are used to store `parameters`, `settings` and `preferences` for software applications.
- They let you adjust robot behavior (sensor limits, PID gains, topic names, frame IDs, robot dimensions, map paths, etc.) without recompiling
- ROS2 uses them heavily with nodes via the --ros-args --params-file option, so you can tune parameters at runtime.

**Launch files**:
- Launch files are XML or Python files that allow you to start multiple nodes and set their parameters in a single command.
- They are especially useful for complex systems with many nodes and parameters.

`In .xml file nested comments arent allowed!`
so use   <ignore_block> </ignore_block>   to comment out blocks of code 




---


**PID**

PID gains are the three tuning parameters of a PID controller:
- P (Proportional gain) → corrects error based on its size (bigger error → stronger correction).
- I (Integral gain) → eliminates steady-state error by accumulating past error.
- D (Derivative gain) → predicts future error by reacting to the rate of change, damping oscillations.

They are helpful in mobile robotics for precise motion control:
- Keeping wheel speeds at the commanded velocity.
- Ensuring smooth trajectory tracking.
- Stabilizing heading or position when following paths.

**`Example`**: In a differential drive robot, PID gains ensure each wheel reaches and maintains the desired RPM despite friction, load, or uneven surfaces.

**Transient vs Steady State**

- Before steady state → when a command is first given (e.g., “wheel at 1 m/s”), the robot is in a transient state. The speed ramps up, may overshoot or oscillate depending on gains.
- Steady state → after some time, the wheel settles close to the target speed and stays there. The error becomes very small (ideally zero).
- After steady state changes → if conditions shift (robot climbs a slope, carries extra weight, battery voltage drops), the system temporarily leaves steady state, and the controller must react to reach the new steady state again.

---



**`Role of robot_state_publisher`**
- It publishes the TF tree of your robot → how all links/joints move relative to each other.
- Input:
   - Your URDF (from robot_description param).
   - Joint states (/joint_states).
- Output:
   - TF frames (/tf and /tf_static).
- Without it:
- RViz can’t visualize robot links correctly.
- Other ROS nodes can’t know where sensors/arms/wheels are.
- Adding the camera URDF update was essential so the camera frame appeared in TF tree.


**`Role of ros_gz_bridge`**
- Gazebo (Ignition/GZ Sim) speaks Gazebo Transport messages(gz.msgs.*).
- ROS2 speaks ROS2 messages (sensor_msgs, geometry_msgs).
- They are different protocols.
- ros_gz_bridge maps one to the other by parameter gz_bridge_node of ros_gz_bridge package.
- Example:
   - /camera/image in Gazebo → bridged → /camera/image in ROS2.
   - /cmd_vel in ROS2 → bridged → /cmd_vel in Gazebo (to control robot).
- Without it:
   - Your sensors (camera, lidar, IMU) would only exist inside Gazebo → ROS2 nodes (like RViz or SLAM) couldn’t use them.
   - Your control commands in ROS2 couldn’t reach the robot in simulation.
   - [link to add more topics:](https://github.com/gazebosim/ros_gz/tree/ros2/ros_gz_bridge)
---

**Sensors**

Sensors are devices that measure physical properties (distance, light, sound, temperature, etc.) and convert them into signals that can be read by a robot's computer.
- Common sensors in robotics include:
  - LIDAR (Light Detection and Ranging) for mapping and obstacle detection.
  - Cameras for vision and object recognition.
  - IMUs (Inertial Measurement Units) for orientation and motion tracking.
  - GPS for outdoor localization.
  - Ultrasonic sensors for distance measurement.
  - Encoders for measuring wheel rotation and speed.

If it's unclear what kind of properties are available for which sensors, the official SDF [reference manual](http://sdformat.org/spec?ver=1.6&elem=sensor#camera_depth_camera) and Documentation for [SDF Syntax with code](http://sdformat.org/tutorials?) is available. 

---

Camera 

- Camera is represented by a small cube. Represented by a fixed joint attched to base link. 
- Another joint names `camera_link_optical` is connected to `camera_link` which is additional link &joint to`solve conflict` between `2 different coordinates systems`
    - By default, URDF uses `right-handed coordinate system` with X forward, Y left, and Z up.
    - However, many ROS drivers and vision processing pipelines expect a camera’s optical axis to be aligned with Z forward, X to the right, and Y down.


In [ ]:
ros2 param set /robot_state_publisher robot_description "$(xacro $(ros2 pkg prefix gazebo_sensors)/share/gazebo_sensors/urdf/mob_bot_mecanum.urdf)"

- `ros2 param set` → tells a running node to update one of its parameters.
- `/robot_state_publisher` → the node we target.
- `robot_description` → the key parameter holding the URDF string.
- `"$(xacro ...)"` → expands the URDF (runs xacro to generate plain XML).
- That whole URDF is then injected directly into the running robot_state_publisher.

Why we do this?
- To reload the robot model on the fly without restarting nodes.
- Helps when debugging URDF/Xacro edits (like adding a sensor, changing joint names, fixing links).

Why here in Camera?:
- We added a camera link/joint to your robot URDF (red cube visual).
- Gazebo spawned it, so visually you saw the cube.
- But ROS2 did not know about the new link/joint because the robot_state_publisher was still running with the old URDF.
- Result → rqt_graph/rqt_topic showed no camera topics.

Camera Image Topic
- Open rqt and select plugins->topic->Monitor and select /camera/image topic.
- Then go again to Plugins->Visualization->Image View to see the camera image.
- Or run directly in terminal 
```bash
ros2 run rqt_image_view rqt_image_view
```


[Next Issue]
Its consuming 6-20 MB bandwidth which is unacceptible for a mobile bot.There is a dedicated `image_bridge` node in the `ros_gz_image` package which can compress mages and make it suitable for mobile robots with KBandwidth constraints.B
- Install to use this: `sudo apt install ros-jazzy-compressed-image-transport`
- Add the node in launch file

[New Issue]
RViz always expect the image and the camera_info topics with the same prefix which works well for: `/camera/image` → `/camera/camera_info` But doesn't work for: `/camera/image/compressed` → `/camera/image/camera_info`. remaping topic still wont resolve this issue as uncompressed image wont work in RViz so use another tool:
- the `relay node` from the `topic_tools package`.
- Relay node to republish `/camera/camera_info` to `/camera/image/camera_info` add node in launch file and dont forgot to add in launchDescription

In [ ]:
# do changes in urdf, and dont relaunch again and again, just run this  and things get updated!

ros2 param set /robot_state_publisher robot_description "$(cat ~/gz_sensors_ws/gazebo_sensors/urdf/mob_bot_mecanum.urdf)"

`rqt_reconfigure` is generally used for dynamically reconfiguring parameters of ROS nodes, including those related to sensors. This can be particularly useful for adjusting camera properties like image resolution, update rate, etc. on the fly.

In [ ]:
# But how do we know what is the name of the parameter 
# and what other settings do we can change? 
# To see that we will use the rqt_reconfigure node.
# launch executables first!, Then start rqt_reconfigure:
ros2 run rqt_reconfigure rqt_reconfigure

---

RQT Graph/Topic

![rqt](../assets/rqt.png)

Explanation:  Gazebo → Bridge → ROS2 Flow

1. Simulation side (Gazebo)

- **Gazebo world + robot** → runs the physics and sensors.  
- **Files**:
  - `.gazebo` → config for plugins (camera, lidar, joints).
  - `.urdf` / `.xacro` → robot description (links, joints, sensors).
- **Role**: Together, these tell Gazebo what the robot is and how it behaves.  
Example: your camera sensor in `.gazebo` file produces `gz.msgs.Image`, not directly a ROS topic.


2. ros_gz_bridge (the translator node)

- Node: `/ros_gz_bridge`
- **Function**: Bridges Gazebo Transport ↔ ROS 2 topics.
- **Examples**:
  - `/camera/image` → `sensor_msgs/msg/Image`
  - `/odom` → `nav_msgs/msg/Odometry`
  - `/joint_states` → `sensor_msgs/msg/JointState`
  - `/tf` → `tf2_msgs/msg/TFMessage`

This makes Gazebo data visible to ROS 2 nodes.


3. ROS2 side (nodes + topics)

- **/robot_state_publisher**
  - Reads `robot_description` (URDF/Xacro XML) + `/joint_states`.
  - Publishes **TF tree** (frames: `base_link → camera_link`).
  - Needed for RViz + Nav2.

- **/trajectory_server & /trajectory**
  - Custom node in your package.
  - Consumes trajectory commands and interacts with robot via TF + odometry.

- **/tf + /robot_description**
  - `/robot_description` = URDF pushed as parameter (from launch file).
  - `/tf` = live coordinate transforms.

- **/cmd_vel**
  - Velocity commands from teleop/navigation/trajectory server.
  - Bridged → robot moves in Gazebo.

- **/camera/image + /camera/camera_info**
  - Data from Gazebo camera plugin.
  - Bridged → ROS topics.
  - Usable in RViz or perception nodes.

4. Where each file fits

- **URDF/Xacro** (`robot.urdf.xacro`)
  - Defines robot structure, joints, sensors.
  - Consumed by `robot_state_publisher`.

- **Gazebo files** (`.gazebo` / `.sdf`)
  - Define simulation-specific plugins (camera noise, lidar range).
  - Consumed by **Gazebo**, not ROS directly.

- **Launch files** (`.launch.py`)
  - Start everything:
    - Spawn robot in Gazebo.
    - Start `ros_gz_bridge`.
    - Start `robot_state_publisher`.
    - Push `robot_description` parameter.

5. Pipeline Flow

  - **Gazebo** runs robot with physics + sensors (URDF + plugins).
  - Sensors + states → produced in Gazebo (raw Gazebo msgs).
  - **ros_gz_bridge** translates → ROS 2 topics.
  - **robot_state_publisher** builds TF tree from URDF + joint states.
  - ROS 2 ecosystem (RViz, SLAM, Nav2, trajectory servers) uses `/odom`, `/camera/image`, `/tf`, `/cmd_vel`.
  - ROS nodes publishing `/cmd_vel` → bridged back to Gazebo → robot moves.


---



In [ ]:
  <!--Camera model in .gazebo file-->
  <gazebo reference="camera_link"><!--refer to link we defined in URDF-->
    <sensor name="camera" type="camera">
      <camera> <!--define below camera related settings-->
        <horizontal_fov>1.3962634</horizontal_fov><!--camera's field of view here in radians-->
        <image>
          <width>640</width>
          <height>480</height>
          <format>R8G8B8</format><!--RGB 8bit pixel format-->
        </image>
        <clip><!--min-max distance camera can see-->
          <near>0.1</near><!--0.1meter-->
          <far>15</far>
        </clip>
        <noise><!--model real world imperfections-->
          <type>gaussian</type>
          <!-- Noise is sampled independently per pixel on each frame.
               That pixel's noise value is added to each of its color
               channels, which at that point lie in the range [0,1]. -->
          <mean>0.0</mean>
          <stddev>0.007</stddev>
        </noise>
        <optical_frame_id>camera_link_optical</optical_frame_id>
        <camera_info_topic>camera/camera_info</camera_info_topic><!--topic publishing intrinsic/extrinsic calibration info-->
        <!--tools like rviz requires a camera_info topic that describes the physical properties of the camera. The topic's name must match camera's topic (in this case both are camera/...camera topic defined below-->
      </camera>
      <always_on>1</always_on>
      <update_rate>20</update_rate><!--Sensor data publisher 20 times per sec (Hz)-->
      <visualize>true</visualize><!--sensor output view in gazebo enabled-->
      <topic>camera/image</topic><!--ROS/Gazebo topic where camera images are published-->
      <!--from SDformat.org-->
      <!--<save>
        <enabled></enabled>
        <path></path>
      </save>-->
    </sensor>
  </gazebo>


  <!--wide angle camera model in .gazebo file-->
  <!--Wide angle Camera-->
  <!--
  <gazebo reference="camera_lens_link">
    <sensor name="wideangle_camera" type="wideanglecamera">
      <camera>
        <horizontal_fov>3.14</horizontal_fov>
        <image>
          <width>640</width>
          <height>480</height>
        </image>
        <clip>
          <near>0.1</near>
          <far>15</far>
        </clip>
        <optical_frame_id>camera_link_optical</optical_frame_id>
        <camera_info_topic>camera/camera_info</camera_info_topic>
      </camera>
      <always_on>1</always_on>
      <update_rate>20</update_rate>
      <topic>camera/image</topic>
      <gz_frame_id>camera_link</gz_frame_id>
    </sensor>
  </gazebo>
  -->


  

In [ ]:
<!--Old Camera Working code-->
 <!-- STEP 7 - Camera -->
  <joint type="fixed" name="camera_joint">
    <origin xyz="0.225 0 0.075" rpy="0 0 0"/>
    <child link="camera_link"/>
    <parent link="base_link"/>
    <axis xyz="0 1 0" />
  </joint>

  <link name='camera_link'>
    <pose>0 0 0 0 0 0</pose>
    <inertial>
      <mass value="0.1"/>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <inertia
          ixx="1e-6" ixy="0" ixz="0"
          iyy="1e-6" iyz="0"
          izz="1e-6"
      />
    </inertial>

    <collision name='collision'>
      <origin xyz="0 0 0" rpy="0 0 0"/> 
      <geometry>
        <box size=".04 .06 .04"/>
      </geometry>
    </collision>

    <visual name='camera_link_visual'>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <geometry>
        <box size=".04 .06 .04"/>
      </geometry>
      <material name="grey"/>
    </visual>

    <visual name='camera_link_visual_lens'>
      <origin xyz="0.032 0 0" rpy="0 1.57 0"/>
      <geometry>
        <cylinder radius="0.015" length="0.027"/>
      </geometry>
      <material name="black"/>
    </visual>
  </link>

  <joint type="fixed" name="camera_optical_joint">
    <origin xyz="0.25 0 0" rpy="-1.5707 0 -1.5707"/>
    <child link="camera_link_optical"/>
    <parent link="camera_link"/>
  </joint>

  <link name="camera_link_optical">
  </link>


  <!-- STEP 7 - Camera -->
  <!--new-->
  <joint type="fixed" name="camera_joint">
    <origin xyz="0.220 0 0.075" rpy="0 0 0"/>
    <parent link="base_link"/>
    <child  link="camera_link"/>
  </joint>

  <link name="camera_link">
    <inertial>
      <mass value="0.1"/>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <inertia ixx="1e-6" ixy="0" ixz="0" iyy="1e-6" iyz="0" izz="1e-6"/>
    </inertial>

    <!-- body box that is only decorative and collision -->
    <collision name="camera_body_collision">
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <geometry><box size="0.04 0.03 0.03"/></geometry>
    </collision>

    <visual name="camera_body_visual">
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <geometry><box size="0.04 0.06 0.04"/></geometry>
      <material name="grey"/>
    </visual>
  </link>

  <!-- separate lens link positioned forward of the box -->
  <joint type="fixed" name="camera_lens_joint">
    <parent link="camera_link"/>
    <!-- put lens where you want the optical center to be -->
    <origin xyz="0.025 0 0" rpy="0 0 0"/>
    <child  link="camera_lens_link"/>
  </joint>

  <link name="camera_lens_link">
    <!-- lens visual placed at this link origin -->
    <visual name="camera_lens_visual">
      <origin xyz="0 0 0" rpy="0 1.5707 0"/>
      <geometry>
        <cylinder radius="0.015" length="0.03"/>
      </geometry>
      <material name="black"/>
    </visual>
  </link>

  <!-- optical frame now coincides with lens link origin -->
  <joint type="fixed" name="camera_optical_joint">
    <parent link="camera_lens_link"/>
    <child  link="camera_link_optical"/>
    <!-- optical joint placed at lens center so sensor uses lens link as reference -->
    <origin xyz="0 0 0" rpy="-1.5707 0 -1.5707"/>
  </joint>

  <link name="camera_link_optical"></link>

Solve header issue of wide angle camera

In [ ]:
ros2 run bme_gazebo_sensors_py image_republisher

---

IMU

An Inertial Measurement Unit (IMU) typically consists of:  
- a 3-axis **accelerometer** 
    - measure `linear acceleration (X,Y,Z)` (rate of change of velocity along x,y,z axis)
    - detect gravity, bumps,vibrations. 
    - Also used to estimate orientation
- a 3-axis **gyroscope**
    - Measures `angular velocity(roll,pitch,yaw)` around X, Y, Z axes.
    - These correspond to roll, pitch, yaw rates (how fast you rotate, or turn)
- a 3-axis **magnetometer**
    - Measures the `Earth’s magnetic field vector (yaw)`.
    - Used as a digital compass → helps estimate absolute heading (yaw/orientation).
    - Measures facing North vs facing East, etc.


    IMU measures linear acceleration, angular velocity, and possibly magnetic heading (orientation). It's important to remember that it's `not possible` to `measure uniform motion` with an IMU where the `velocity is constant` (`acceleration is zero`) and there is `no change in the orientation`. Therefore we cannot replace the odometry of the robot with an IMU but with the right technique we can combine these two into a more precise measurement unit.

*** Always add `topics` of plugins defined in .gazebo file to the bridge node in launch file to make them available in ROS2.

---

**Odometry**:
It's estimating a robot’s position (x, y) and orientation (θ/yaw) over time by integrating motion information from its sensors.
For a differential drive robot (two wheels) example:
- Each wheel has an encoder that counts how many ticks it rotated.
- From encoder ticks → we get distance traveled by each wheel.

Odometry Sources
- Wheel encoders (most common) → measure wheel rotations.
- Visual odometry (camera-based, e.g. ORB-SLAM) → track visual features.
- Lidar odometry (scan matching).

**Drift**
- Drift in wheel encoders = the cumulative error in odometry as the robot moves.
- It happens because encoders only measure wheel rotations, not actual ground motion.
- If wheels slip (e.g., on smooth floor, turning, or uneven ground), ticks still increase but the robot hasn’t moved that distance.
- Over time, these small errors add up → robot’s `estimated position` `drifts away` from its `true position`

**Need of `Sensor Fusion`?**

Reason: IMU's cant measure constant velocity motion (zero acceleration) and odometry drifts over time due to wheel slip, uneven terrain, etc.
- We combine IMU + odometry using sensor fusion (e.g., an Extended Kalman Filter, EKF).
- Odometry gives long-term position tracking but drifts with wheel slip.
- IMU gives short-term motion detection but can’t see constant motion.
- Together → more precise state estimation.



---
Plugins in World Files For simulation

In [ ]:
# add in world.sdf files:

    <plugin name='gz::sim::systems::Physics' filename='gz-sim-physics-system'/>
    <plugin name='gz::sim::systems::UserCommands' filename='gz-sim-user-commands-system'/>
    <plugin name='gz::sim::systems::SceneBroadcaster' filename='gz-sim-scene-broadcaster-system'/>
    <plugin name='gz::sim::systems::Contact' filename='gz-sim-contact-system'/>
    <plugin
      filename="gz-sim-sensors-system"
      name="gz::sim::systems::Sensors">
      <render_engine>ogre2</render_engine>
    </plugin>
    <plugin
      filename="gz-sim-imu-system"
      name="gz::sim::systems::Imu">
    </plugin>
    <plugin
      filename="gz-sim-navsat-system"
      name="gz::sim::systems::NavSat">
    </plugin>

---

1. **Linear Systems**

A linear system means its equations obey `superposition`:
- If input A → output X, then input B → output Y,  
- Then input (A + B) → output (X + Y). (`Additivity` Property)
- If input A → output X, then input (k*A) → output (k*X). (`Homogeneity` Property)
- Example:  x(k+1) = x(k) + v * Δt  
> Velocity v just scales linearly with time.  
> Graph: `straight line relation` (no curves, no trig).  
> Linear Kalman Filter works perfectly here.  

2. **Non-linear Systems**

A non-linear system breaks superposition: doubling the input does not double the output.  
Equations involve `sin`, `cos`, `products of states`, `division` etc.  
- Example: Robot orientation update:  
    - x(k+1) = x(k) + v * cos(θ) * Δt  
    - y(k+1) = y(k) + v * sin(θ) * Δt  
>Here motion depends on cos(θ), sin(θ) → non-linear functions.  
>If robot turns by 90°, cos and sin change abruptly.  
>A linear model can’t handle this curvature.  

3. Why **Linear KF fails**

Linear KF assumes system dynamics can be written as:  
- x(k+1) = A x(k) + B u(k) + w(k)  (Motion model)
- z(k) = H x(k) + v(k)             (Measurement model)
where A, B, H are matrices (linear transforms).  
- But real robot motion (turning, IMU readings, GPS with Earth curvature) → involves sin, cos, quaternions.If we force-fit them into a linear model:  
    - Predictions diverge (robot thinks it’s straight when it’s curving).  
    - Measurements mismatch badly.  

4. **Extended Kalman Filter (EKF)**

EKF is a fix for this: it can work with non-linear dynamics and measurements. `Key idea: Local linearization` 
- Take your non-linear equations:  
    - x(k+1) = f(x(k), u(k)) + w(k)  (Motion model) 
    - z(k) = h(x(k)) + v(k)          (Measurement model)

where f and h are non-linear (sin, cos, etc).  

- EKF approximates them as linear near the current estimate using Jacobian matrices (derivatives).  
- So instead of globally linear, EKF says:  
   - “Okay, around this small region, motion is almost linear.”  

5. `***EKF Steps***`
- `Prediction` (non-linear motion model): Use f(x,u) (with cos, sin, etc.) to predict where robot should be.  
- `Linearize` (Jacobian): Compute derivatives of f and h at current state → gives you local linear models.  
- `Update` (non-linear measurement model): Compare predicted measurement (from IMU, GPS, encoders) with actual one. Update estimate with corrected covariance.  

6. Example
- Suppose robot drives forward with velocity v and orientation θ.  Linear KF would say:  
    - x(k+1) = x(k) + v * Δt   (always in a straight line).  
- Real robot (non-linear):  
    - x(k+1) = x(k) + v * cos(θ) * Δt  
    - y(k+1) = y(k) + v * sin(θ) * Δt  
- If θ = 90° → robot moves upwards, not straight. KF would fail here. EKF handles it by linearizing cos(θ), sin(θ) locally → so update works.  

---

**EKF**  (State Estimation using Non linear Motion model, Global nonlinear becomes locally linear using Jacoboian, Update nonlinear model using predicted vs actual measurement from sensors)

An Extended Kalman Filter (EKF) algorithm is used to estimate vehicle `position`, `velocity` and `angular orientation` based on rate gyroscopes, accelerometer, compass, GPS, airspeed and barometric pressure measurements.

- The `advantage` of the EKF over the simpler complementary filter algorithms (i.e. “Inertial Nav” or DCM), is that by fusing all available measurements it is better able to `reject measurements with significant errors`

Background:

Unlike the standard Kalman Filter, which is designed for linear systems, the EKF can handle `non-linear models`. Real-world systems, such as the movement of a car or a person, rarely follow perfectly linear paths. The EKF addresses this by using a mathematical technique called `linearization`. At each time step, it` approximates the non-linear system` with a `linear one` around the `current state estimate` using `*Jacobian`. This allows it to apply the same `predict-and-update cycle` as the standard Kalman Filter.

The cycle works as follows:
- `Prediction`: The EKF predicts the system's next state based on its current estimate and a motion model. This prediction inherently includes some uncertainty.
- `Update`: The filter then incorporates a new measurement from a sensor (like a camera or GPS). It compares the actual measurement to the predicted measurement to calculate a correction, which is then used to update and refine the state estimate.

[EKF explained simply link](https://simondlevy.github.io/ekf-tutorial/) (Mathematics + Code + Examples)

---

EKF (Must be understood for interview) **

1. General EKF Equations

`Motion (Process) Model:`
- x(k+1) = f(x(k), u(k)) + w(k)

    - x(k): State vector at time step k (what we want to estimate, e.g. robot pose [x, y, θ]).
    - u(k): Control input at time step k (e.g. wheel velocities, commanded velocity).
    - f(⋅): Nonlinear function describing how the state evolves (robot motion equations).
    - w(k): Process noise (uncertainty in motion, e.g. wheel slip).

`Measurement (Observation) Model`:
- z(k) = h(x(k)) + v(k)

    - z(k): Measurement at time step k (e.g. GPS coordinates, range to landmark).
    - h(⋅): Nonlinear function mapping state → sensor space.
    - v(k): Measurement noise (sensor uncertainty).

2. Example Problem — Mobile Robot with GPS

- `Robot State`:
    - x(k) = [x, y, θ]
- `Control Input`: forward velocity `v`, angular velocity `ω`.
- Motion Model (differential drive kinematics):
- f(x(k), u(k)) =
    [
    x + v cos(θ) Δt ; 
    y + v sin(θ) Δt ; 
    θ + ω Δt
    ]
- Measurement Model (GPS):
    - GPS directly measures position (not orientation):
        - h(x(k)) = [x, y]


3. Numerical Example

- Initial state estimate:
    - x0 = [2, 3, 45°]; (robot at (2,3) facing NE).

- Control input:
    - v = 1.0 m/s (forward)
    - ω = 0 rad/s (straight)
    - Δt = 1 s
- Process Model:
   - x1 =
    [
    2 + 1⋅cos(45°)⋅1 ;
    3 + 1⋅sin(45°)⋅1 ;
    45° + 0 ;
    ]
    =
    [2.707, 3.707, 45°]. So the predicted state is (2.71, 3.71, 45°).

- Measurement: GPS reports
    - z1 = [2.9, 3.6]
- Measurement Model:
    - h(x1) = [x, y] = [2.707, 3.707]
- Innovation (measurement error):
    - z − h(x) = [2.9, 3.6] − [2.707, 3.707] = [0.193, -0.107]. So GPS says “you’re 0.193 m more east, 0.107 m more south than your motion model predicted.”


4. What EKF Does With This

- Prediction Step:
    - Uses f(x,u) to move state forward.
    - Uses Jacobian F = ∂f/∂x to propagate uncertainty.
- Correction Step:
    - Compares predicted measurement h(x) with real sensor reading z.
    - Uses Jacobian H = ∂h/∂x to figure out how measurement error relates to state error.
    - Updates state estimate:
        - x(new) = x(pred) + K(z − h(x(pred))). where K is the Kalman gain (balance between trusting prediction vs measurement).

---


![alt text](assets/k1.png)
<img src="assets/k2.png" alt="alt text" width="400" height="630">
![alt text](assets/k3.png)

Role of Jacobians in EKF (with the same GPS robot example)

1. Why Jacobians are needed
- Our motion model is nonlinear:
    - f(x,u) = [
    x + v cos(θ) Δt;
    y + v sin(θ) Δt;
    θ + ω Δt;
    ]
- The terms cos(θ) and sin(θ) make it nonlinear.
- Linear Kalman Filter requires state updates of the form A x + B u (purely linear).
- Solution in EKF: locally approximate these nonlinear equations with a linear model around the current estimate. 
- This is done using Jacobians (matrices of partial derivatives).

2. Jacobian of Motion Model
- State: x = [x, y, θ]
- Control: u = [v, ω]
- Motion update:
   - x' = x + v cos(θ) Δt
   - y' = y + v sin(θ) Δt
   - θ' = θ + ω Δt

- Jacobian F = ∂f/∂x.  Take partial derivatives of each new state wrt old state [x, y, θ].

    - ∂x'/∂x = 1
    - ∂x'/∂y = 0
    - ∂x'/∂θ = -v sin(θ) Δt
    .....
    - ∂y'/∂x = 0
    - ∂y'/∂y = 1
    - ∂y'/∂θ = v cos(θ) Δt
    .....
    - ∂θ'/∂x = 0
    - ∂θ'/∂y = 0
    - ∂θ'/∂θ = 1
    .....
- So
    - F =
    [ 1   0   -v sin(θ) Δt
    0   1    v cos(θ) Δt
    0   0    1 ]
- This tells how small changes in x,y,θ affect the next state near current θ.

3. Jacobian of Measurement Model
- Measurement model: h(x) = [x, y]
- We only measure position.

    - ∂h/∂x = 1   0   0;
              0   1   0. So H =[1 0 0;
          0 1 0]

4. Numerical Example at θ = 45°
- Let v=1, Δt=1.

    - F =
    [1   0   -1⋅sin(45°)⋅1;
    0   1    1⋅cos(45°)⋅1;
    0   0    1]
    = 
    [1   0   -0.707;
    0   1    0.707;
    0   0    1]

- This means:
    - A small error in θ produces about -0.707 m change in x and +0.707 m change in y in the next step.
    - Exactly what you'd expect: if you’re slightly wrong about θ, your x/y prediction shifts diagonally.

5. Why this works
Globally, motion is nonlinear because cos(θ), sin(θ) curve.
But around θ=45°, we linearize using the Jacobian → straight-line approximation.
Next EKF step: use this local linear form to update covariance and combine measurements.
At each step, EKF recomputes Jacobians → always linearizing around the latest pose.
This converts global nonlinear dynamics into locally linear updates that the Kalman filter machinery can handle.

![alt text](assets/k4.png)
<img src="assets/k5.png" alt="alt text" width="560" height="665">

---

In [ ]:
# Robot localization package in ROS
sudo apt install ros-jazzy-robot-localization


1. Role of TF in ROS  
• TF keeps track of coordinate frames (map, odom, base_link, camera_link, etc).  
• A transform is "frame A relative to frame B".  
• Example: odom → base_link shows robot pose in odometry frame.  

2. TF publishers in this case  
• ros_gz_bridge: bridges Gazebo poses, publishes odom → base_link if /tf is bridged.  
• robot_state_publisher: publishes static transforms from URDF (relationship btw base_link → sensors, wheels).  

3. The problem  
• robot_localization also publishes odom → base_link (filtered odometry).  
• Gazebo and robot_localization both publishing the same transform causes conflicts.  
• TF does not allow multiple sources for one transform.  

4. The solution  
• Keep only one truth for odom → base_link.  
• Trust robot_localization (fused odometry).  
• Stop Gazebo from publishing TF by commenting out:  
  "/tf@tf2_msgs/msg/TFMessage@gz.msgs.Pose_V"  

5. Summary  
• ros_gz_bridge: raw Gazebo poses (disable TF part).  
• robot_state_publisher: static transforms from URDF.  
• robot_localization: fused odometry (final odom → base_link).  
• Disable Gazebo’s TF bridge → avoid duplicate odometry transform.  


In [ ]:
# See no info from rqt_tf_tree, but see ros_gz pusblisher using topic info.
ros2 launch gazebo_sensors spawn_robot.launch.py

ros2 run rqt_tf_tree rqt_tf_tree

ros2 topic info /tf --verbose

We can see that the yellow (raw) odometry starts drifting away from the corrected one very quickly and we can easily bring the robot into a `special situation` if we drive on a curve and hit the wall. In this case the robot is unable to move and the wheels are slipping. The raw odometry believes from the encoder signals that the robot is still moving on a curve while the odometry after the ekf sensor fusion will believe that the robot moves forward straight. Although none of them are correct, but remember, `neither the IMU and neither the odometry` can tell if the robot is doing an `uniform movement` or it's `stand still`. At least the ekf is able to properly tell that the robot's orientation is not changing regardless what the encoders measure.

---

Navigation's Concepts for GPS



**Latitude (φ) and Longitude (λ)**
- Latitude (φ): north–south position (−90° = South Pole, 0° = Equator, +90° = North Pole). y-axis
- Longitude (λ): east–west position (−180° to +180°). x-axis

**Haversine**
- This combines latitude/longitude differences into a value proportional to the chord length between the two points (straight line through Earth).

**Central Angle C**
- What: angle at the Earth’s center between the two locations.

- Why useful: multiplying this angle by Earth’s radius 𝑅 gives the `arc length` (distance along Earth’s surface).
That’s the shortest path (great-circle distance) between two points on a sphere.

- Formula:
    - a = sin²(Δφ/2) + cos(φ1) * cos(φ2) * sin²(Δλ/2) (haversine)
    - c = 2 * atan2(√a, √(1−a)) (central angle)
    - d = R * c (arc length = distance)
Where:
- φ1, φ2 = latitudes of point 1 and 2 (in radians)
- λ1, λ2 = longitudes of point 1 and 2 (in radians)
- Δφ = φ2 − φ1
- Δλ = λ2 − λ1
- R = Earth’s radius (mean radius = 6,371 km)

**Bearing (Initial Heading, θ)**
- The initial heading you must take at Point 1 to travel along the shortest path (great circle) to Point 2.
- Units: degrees (0° = North, 90° = East, 180° = South, 270° = West).
- Used in navigation systems, autopilot, robotics for orientation.

Here Arc means inverse trigonometry; arcsin(0.5)=30∘, because sin(30∘)=0.5


Summary:
- The `Haversine formula` computes `great-circle distance` (shortest path on a sphere) from latitude/longitude, which is more accurate than flat-Earth approximations.
- The `central angle` gives a numerically stable way to derive `arc length`, even for very small or very large distances.
- The initial `bearing` (azimuth) provides the correct navigation `heading` between two global coordinates.
- These calculations are efficient (just trig + sqrt), making them practical for GPS, GIS, aviation, shipping, and robotics.

The `haversine + bearing` method is still standard today in navigation and mapping software, though higher-precision ellipsoidal models (like Vincenty) are sometimes used for long distances.

![alt text](assets/g1.png)
![alt text](assets/g2.png)

In [ ]:
# to view gps mapping

ros2 launch gazebo_sensors spawn_robot.launch.py world:=empty.sdf rviz_config:=gps.rviz x:=0.0 y:=0.0 yaw:=0.0


# to navigate gps between 4 points and then stopping:
ros2 run gazebo_sensors_py gps_waypoint_follower

---



**Lidar**

According to the different ranging methods, LiDAR is mainly divided into ToF (Time of Flight) and FMCW (Frequency Modulated Continuous Wave) types, and ToF type is the vast majority of LIDARs currently in mass production.

1. In the `ToF` (Time of Flight) method (OS1), the LiDAR transmitter emits a pulse, hits an object and returns, and the receiver receives the return wave and calculates the difference in reception time between the two, and multiplies it by the speed of light to achieve distance measurement between objects.
    - d=c⋅t​/2
    - Where d is the distance between the object and the LiDAR, c is the speed of light, and t is the round-trip time of the pulse (ToF).
    - Pros: simple principle, long range (up to hundreds of meters).
    - Cons: lower accuracy (cm level), timing precision limits accuracy

2. Phase-Shift LiDAR
Emit continuous modulated laser (sinusoidal intensity). Measure phase difference Δ𝜙 between sent and received signal.
    - Distance: 𝑑=Δ𝜙⋅𝜆/4𝜋
    - Where 𝜆 is the modulation wavelength (speed of light / modulation frequency). 
    - Pros: very accurate for short–medium range.
    - Cons: max range limited (phase ambiguity).

3. Coherent methods are also used, i.e., for `frequency modulated continuous wave` (FMCW: Aeva, Aurora) LiDAR transmitting a continuous beam with a frequency that varies steadily over time. Since the frequency of the source beam is constantly changing, differences in beam transmission distance result in differences in frequency. After mixing the echo signal with the local oscillation signal and low-pass filtering, the resulting differential frequency signal is a function of the beam round-trip time. The FM continuous wave LiDAR will not be interfered by other LiDAR or sunlight and has no range blindness; it can also measure the speed and distance of objects using Doppler shift. The FM continuous wave LiDAR concept is not new, but faces a number of technical challenges, such as the linewidth limitation of the emitted laser, the frequency range of the linear FM pulses, the linearity of the linear pulse frequency variation, and the reproducibility of individual linear FM pulses.
    - Emit chirped (linearly frequency-swept) laser.
    - Beat frequency Δf between sent and received gives delay → distance:𝑑=𝑐⋅Δ𝑓/2⋅𝑘
    - Where 𝑘 = chirp rate.
    - Bonus: Doppler shift also gives velocity.
    - Pros: range + velocity, very resistant to interference.
    - Cons: complex, costlier, still emerging.




<img src="assets/l1.png" alt="alt text" width="500" height="400">
<img src="assets/l2.png" alt="alt text" width="500" height="400">

Gazebo with the `Visualize Lidar` tool from plugins.
- Increase decay time of visualization of lidar scans to do a simple mapping of environment. (from 0 to 30)

![alt text](assets/l3.png)

3D Lidar

If we want to simulate a 3D lidar we only have to `increase` the `number of vertical samples` together with the minimum and maximum angles. For example the following vertical parameters are matching a Velodyne VLP-32 sensor:

In [ ]:
          <vertical>
              <samples>32</samples>
              <min_angle>-0.5353</min_angle>
              <max_angle>0.1862</max_angle>
          </vertical>

To properly visualize a 3D point cloud in RViz we have to forward one more topic with parameter_bridge:

In [ ]:
"/scan/points@sensor_msgs/msg/PointCloud2@gz.msgs.PointCloudPacked",

---

RGBD camera

An RGBD camera combines a standard `RGB camera` with a `depth sensor` to provide both `color images` and `depth information` for each pixel. This allows the camera to capture `3D information` about the environment, which is useful for robotics applications like mapping, navigation, and object recognition.



In [ ]:
 <gazebo reference="camera_lens_link"><!--changed-->
    <sensor name="rgbd_camera" type="rgbd_camera">
      <camera>
        <horizontal_fov>1.25</horizontal_fov>
        <image>
          <width>320</width>
          <height>240</height>
        </image>
        <clip>
          <near>0.3</near>
          <far>15</far>
        </clip>
        <optical_frame_id>camera_link_optical</optical_frame_id>
      </camera>
      <always_on>1</always_on>
      <update_rate>20</update_rate>
      <visualize>true</visualize>
      <topic>camera</topic>
      <gz_frame_id>camera_link</gz_frame_id>
    </sensor>
  </gazebo>

---

Other Adds-on Sensors  ([link](https://github.com/gazebosim/gz-sim/tree/gz-sim8/examples/worlds))

| Sensor                            | Example                                         | Use                                                              |
| --------------------------------- | ----------------------------------------------- | ---------------------------------------------------------------- |
| **Contact sensor**                | `contact_sensor.sdf`                            | Detects physical collisions                                      |
| **Thermal / segmentation camera** | `thermal_camera.sdf`, `segmentation_camera.sdf` | Advanced vision (CV/AI)                                          |
| **Environmental sensor**          | `environmental_sensor.sdf`                      | Temperature, humidity—useful for multi-sensor fusion experiments |


---



Image Processing with OpenCV

- Use conventional camera for opencv tasks!

- If we want to use OpenCV and other python modules from a python virtual environment, we'll have to add the following to the `setup.cfg` file inside our python package:
```bash
[build_scripts]
executable = /usr/bin/env python3
```

- Writing an Open cv node that `subscribes` to /camera/image topic, `converts` it to OpenCV compatible frame and `displays` it using OpenCV's imshow function.



when process_image() function starts doing `heavier image processing` (like OpenCV computations), it’ll take more time to run. If it runs in the `same thread` as `rclpy.spin_once()`, it will `block ROS from receiving new messages` (like the next camera frame).

So, the solution is to move the ROS spin loop into a separate thread, letting one thread handle incoming images continuously while another handles image processing — keeping the node responsive and real-time.

---

In [ ]:
  <!-- STEP 7 - Camera -->
  <!--new after body update, no camera set-->
  
  <joint type="fixed" name="camera_joint">
    <origin xyz="0.220 0 0.075" rpy="0 0 0"/>
    <parent link="base_link"/>
    <child  link="camera_link"/>
  </joint>

  <link name="camera_link">
    <inertial>
      <mass value="0.1"/>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <inertia ixx="1e-6" ixy="0" ixz="0" iyy="1e-6" iyz="0" izz="1e-6"/>
    </inertial>

    <!-- body box that is only decorative and collision -->
    <collision name="camera_body_collision">
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <geometry><box size="0.04 0.03 0.03"/></geometry>
    </collision>

    <visual name="camera_body_visual">
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <geometry><box size="0.04 0.06 0.04"/></geometry>
      <material name="grey"/>
    </visual>
  </link>

  <!-- separate lens link positioned forward of the box -->
  <joint type="fixed" name="camera_lens_joint">
    <parent link="camera_link"/>
    <!-- put lens where you want the optical center to be -->
    <origin xyz="0.025 0 0" rpy="0 0 0"/>
    <child  link="camera_lens_link"/>
  </joint>

  <link name="camera_lens_link">
    <!-- lens visual placed at this link origin -->
    <visual name="camera_lens_visual">
      <origin xyz="0 0 0" rpy="0 1.5707 0"/>
      <geometry>
        <cylinder radius="0.015" length="0.03"/>
      </geometry>
      <material name="black"/>
    </visual>
  </link>

  <!-- optical frame now coincides with lens link origin -->
  <joint type="fixed" name="camera_optical_joint">
    <parent link="camera_lens_link"/>
    <child  link="camera_link_optical"/>
    <!-- optical joint placed at lens center so sensor uses lens link as reference -->
    <origin xyz="0 0 0" rpy="-1.5707 0 -1.5707"/>
  </joint>

  <link name="camera_link_optical"></link>

---


To do:
1. Draw a slide on EKF Sensor fusion, equations, usecase in robotics. Then slide on diagram on Drift in odometry vs filtered ekf odometery (green vs yellow output)
2. Differential Drive Equations slide and Modelling
3. Sensors slide (with IMU, Lidar, Camera, GPS)- 3x slides with 1 focusing on point cloud of lidar and camera depth image
4. Image processing with opencv
